# Air Quality Monitoring

### Import/Install packages

In [1]:
#!pip install git+https://github.com/datakaveri/iudx-python-sdk
#!pip install geojsoncontour
#!pip install voila
#!pip install voila-gridstack

# After running all the cells use 
# !voila airQualityMonitoring.ipynb --enable_nbextensions=True --template=gridstack
# to launch dashboard or use jupyter notebook extensions

from iudx.entity.Entity import Entity

import pandas as pd
import numpy as np
import json
from datetime import date, datetime, timedelta

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium import plugins
from scipy.interpolate import griddata
import geojsoncontour

import ipywidgets as widgets
from ipywidgets import Layout

import warnings

### Defining variables and widgets for interaction

In [3]:
# ids of each resource group
city_ids={
    "vadodara": "vmc.gov.in/ae95ac0975a80bd4fd4127c68d3a5b6f141a3436/rs.iudx.org.in/vadodara-env-aqm",
    "varanasi": "varanasismartcity.gov.in/62d1f729edd3d2a1a090cb1c6c89356296963d55/rs.iudx.org.in/varanasi-env-aqm",
    "pune": "datakaveri.org/04a15c9960ffda227e9546f3f46e629e1fe4132b/rs.iudx.org.in/pune-env-aqm"
}
# types of values measured in each city - instantaneous or average
value_type={
    'vadodara': 'instValue',
    'varanasi': 'avgOverTime',
    'pune':'avgOverTime'
}
# list of properties common to all cities
column_choice=['ambientNoise','so2','uv','co','co2','illuminance','no2','o3','pm10','pm2p5','relativeHumidity']

# widgets for interaction
prompt1=widgets.HTML(value="")
prompt2=widgets.HTML(value="")
gif_address = 'https://www.uttf.com.ua/assets/images/loader2.gif'
select_ndays=widgets.IntSlider(
    value=1,
    min=1,
    max=14,
    step=1,
    description='Days: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
select_city=widgets.Dropdown(
    options=city_ids.keys(),
    value='pune',
    description='City:',
    disabled=False,
)
select_col=widgets.Dropdown(
    options=column_choice,
    value='pm10',
    description='Property:',
    disabled=False,
)
mywidgets=[select_city,select_ndays,select_col]
ui=widgets.VBox([select_city,select_ndays,prompt1,select_col,prompt2])

### Functions to fetch, prepare and visualize data

##### Fetch data

In [4]:
# fetch latest data in the past n days for a city and add/modify required columns
def get_data(selected_city,ndays):
    for widget in mywidgets:
        widget.disabled=True
    prompt1.value=f'<img src="{gif_address}" height=150 width=150> Fetching data'
    global entity,measures,latest_measures,start_time,end_time,city
    city=selected_city
    entity=Entity(entity_id=city_ids[city])
    latest_measures=entity.latest().reset_index(drop=True)
    end_time = latest_measures['observationDateTime'].sort_values(ascending=False).reset_index(drop=True)[0]
    start_time = (end_time - timedelta(days=ndays,hours=6))
    measures = entity.during_search(
        start_time=start_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
        end_time=end_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
    )
    measures['observationDateTime']=measures['observationDateTime'].apply(lambda x:x.tz_localize(None))
    latest_measures['observationDateTime']=latest_measures['observationDateTime'].apply(lambda x:x.tz_localize(None))
    rs_coordinates={}
    rs_label={}
    for res in entity.resources:
        rs_coordinates[res['id']]=res['location']['geometry']['coordinates']
        rs_label[res['id']]=res['name']
    latest_measures['x_co']=latest_measures['id'].apply(lambda id:rs_coordinates[id][0])
    latest_measures['y_co']=latest_measures['id'].apply(lambda id:rs_coordinates[id][1])
    measures['x_co']=measures['id'].apply(lambda id:rs_coordinates[id][0])
    measures['y_co']=measures['id'].apply(lambda id:rs_coordinates[id][1])
    measures['label']=measures['id'].apply(lambda id:rs_label[id])
    latest_measures['label']=measures['id'].apply(lambda id:rs_label[id])
    for widget in mywidgets:
        widget.disabled=False
    prompt1.value=f'Fetched {measures.shape[0]} records from {len(entity.resources)} resources'

##### Spatial Visualization

In [5]:
# plot contours over a map for a given property
def spatialVis1(selected_city, col):
    global city,units
    prop_desc=entity._data_descriptor[col][value_type[city]]
    units=prop_desc["unitText"]
    prompt2.value=f'{prop_desc["description"]}<br> Unit: {units}'
    city=selected_city
    column_name=col+"."+value_type[city]
    x_orig = []
    y_orig = []
    zs = []
    for res in entity.resources:
        try:
            val = latest_measures[latest_measures["id"] == res["id"]][column_name].values[0]
            if val is not None and val>0:
                zs.append(val)
                x_orig.append(res["location"]["geometry"]["coordinates"][0])
                y_orig.append(res["location"]["geometry"]["coordinates"][1])
        except:
            pass
    x_orig = np.array(x_orig)
    y_orig = np.array(y_orig)
    zs = np.array(zs)
    # Initialize the map
    geomap1 = folium.Map([y_orig.mean(), x_orig.mean()], zoom_start=11, tiles="cartodbpositron")
    for res in entity.resources:
        entity_id = res["id"]
        try:
          val=latest_measures[latest_measures['id']==entity_id][column_name].values[0]
          if val is not None and val>0:
            folium.Marker([res["location"]["geometry"]["coordinates"][1], res["location"]["geometry"]["coordinates"][0]],
                          tooltip=f'{col.upper()}: {str(val)}').add_to(geomap1)
        except:
          pass
    # Make lat and lon linspace
    y_arr = np.linspace(np.min(y_orig), np.max(y_orig), 100)
    x_arr = np.linspace(np.min(x_orig), np.max(x_orig), 100)
    # Make mesh grid
    x_mesh, y_mesh = np.meshgrid(x_arr, y_arr)
    # Perform cubic interpolation
    z_mesh = griddata((x_orig, y_orig), zs, (x_mesh, y_mesh), method='cubic')
    # Number of levels of colors
    levels = 20
    contourf=plt.contourf(x_mesh, y_mesh, z_mesh, levels, alpha=0.5, 
                              cmap="bwr", linestyles='None', vmin=0, vmax=100)
    plt.close()
    # Convert matplotlib contourf to geojson
    geojson = geojsoncontour.contourf_to_geojson(
        contourf=contourf,
        min_angle_deg=3.0,
        ndigits=5,
        stroke_width=1,
        fill_opacity=0.5)
    # Plot the contour plot on folium
    folium.GeoJson(
        geojson,
        style_function=lambda x: {
            'color':     x['properties']['stroke'],
            'weight':    x['properties']['stroke-width'],
            'fillColor': x['properties']['fill'],
            'opacity':   0.6,
        }).add_to(geomap1)
    # Show map
    display(geomap1)

In [6]:
# plot bubbles over a map for a given property
def spatialVis2(selected_city, col):
    city=selected_city
    column_name=col+"."+value_type[city]
    maxval=max(list(filter(None,latest_measures[column_name])))
    minval=min(list(filter(None,latest_measures[column_name])))
    geomap2 = folium.Map([latest_measures['y_co'].mean(), latest_measures['x_co'].mean()], zoom_start=12, tiles="cartodbpositron")
    for res in entity.resources:
        entity_id = res["id"]
        try:
          val=latest_measures[latest_measures['id']==entity_id][column_name].values[0]
          if val is not None and val>0:
            folium.Circle(
              [res["location"]["geometry"]["coordinates"][1], res["location"]["geometry"]["coordinates"][0]],
              radius=2000*(val-minval)/(maxval-minval),
              popup = f'{col.upper()}: {str(val)}',
              color='b',
              fill_color=('red' if ((val-minval)/(maxval-minval))>0.6 else 'blue'),
              fill=True,
              fill_opacity=0.4
              ).add_to(geomap2)
        except:
          pass
    display(geomap2)

##### Time Series Visualization

In [7]:
# plot the measures of a proprty over ndays for the resource with the latest recording
def timeSeriesVis1(selected_city, col, ndays):
    column_name=col+"."+value_type[city]
    sensor_id = measures.sort_values(by='observationDateTime',ascending=False).reset_index(drop=True)['id'][0]
    single_resource_data = measures.query(f"id == '{sensor_id}'")
    sensor_coordinates=[]
    for res in entity.resources:
      if res['id']==sensor_id:
        sensor_coordinates=res['location']['geometry']['coordinates']
    fig = px.line(
        single_resource_data, 
        x="observationDateTime", 
        y=column_name
    )
    display(widgets.HTML(f'<center style="font-size:14px">Temporal sensor reading for \n {col.upper()} from {start_time.date()} to {end_time.date()} for resource at {sensor_coordinates}<center>'))
    fig.update_layout(
        xaxis_title="Observed Timestamp",
        yaxis_title="Sensor reading for "+col.upper()+" ("+units+")",
        font=dict(
            size=12
        )
    )
    fig.update_xaxes(rangeslider_visible=True)
    fig.show()

In [8]:
# plot the measures of a proprty over ndays for all resources
def timeSeriesVis2(selected_city, col, ndays):
    column_name=col+"."+value_type[city]
    fig = px.line(
        measures, 
        x="observationDateTime", 
        y=column_name,
        color='label'
    )
    display(widgets.HTML(f'<center style="font-size:14px">Temporal sensor reading for {col.upper()} from {start_time.date()} to {end_time.date()} of all sensors<center>'))
    fig.update_layout(
        xaxis_title="Observed Timestamp",
        yaxis_title="Sensor reading for "+col.upper()+" ("+units+")",
        font=dict(
            size=12
        )
    )
    fig.update_xaxes(rangeslider_visible=True)
    fig.show()

In [9]:
# plot a box plot over each day of the week for the resource with the latest recording
def timeSeriesVis3(selected_city, col, ndays):
    column_name=col+"."+value_type[city]
    sensor_id = measures.sort_values(by='observationDateTime',ascending=False).reset_index(drop=True)['id'][0]
    single_resource_data = measures.query(f"id == '{sensor_id}'")
    warnings.filterwarnings('ignore')
    sensor_coordinates=[]
    single_resource_data['day']=single_resource_data['observationDateTime'].apply(lambda x:x.strftime('%A'))
    for res in entity.resources:
      if res['id']==sensor_id:
        sensor_coordinates=res['location']['geometry']['coordinates']
    fig = px.box(
        single_resource_data, 
        x="day", 
        y=column_name,
        points="all"
    )
    display(widgets.HTML(f'<center style="font-size:14px">Box plots for \n {col.upper()} from {start_time.date()} to {end_time.date()} for resource at {sensor_coordinates}<center>'))
    fig.update_layout(
        #title=f'',
        xaxis_title="Day",
        yaxis_title="Sensor reading for "+col.upper()+" ("+units+")",
        font=dict(
            size=12
        )
    )
    fig.show()

In [10]:
# plot a histogram showing the average measurements over observed time for the resource with the latest recording
def timeSeriesVis4(selected_city, col, ndays):
    column_name=col+"."+value_type[city]
    sensor_id = measures.sort_values(by='observationDateTime',ascending=False).reset_index(drop=True)['id'][0]
    single_resource_data = measures.query(f"id == '{sensor_id}'")
    warnings.filterwarnings('ignore')
    sensor_coordinates=[]
    single_resource_data['day']=single_resource_data['observationDateTime'].apply(lambda x:x.strftime('%A'))
    for res in entity.resources:
      if res['id']==sensor_id:
        sensor_coordinates=res['location']['geometry']['coordinates']
    fig = px.histogram(
        single_resource_data, 
        x="observationDateTime", 
        y=column_name,
        histfunc="avg"
    )
    display(widgets.HTML(f'<center style="font-size:14px">Histogram for \n {col.upper()} from {start_time.date()} to {end_time.date()} for resource at {sensor_coordinates}<center>'))
    fig.update_layout(
        #title=f'',
        xaxis_title="Day",
        yaxis_title="Sensor reading for "+col.upper()+" ("+units+")",
        font=dict(
            size=12
        )
    )
    fig.show()

##### Basic Visualization

In [11]:
# plot a bar chart for the latest measures of a property at all active resources
def simpleVis1(selected_city, col):
    column_name=col+"."+value_type[city]
    display(widgets.HTML(f'<center style="font-size:14px">Latest temporal sensor reading for {col.upper()} of all sensors<center>'))
    fig = px.bar(latest_measures, x='label', y=column_name)
    fig.update_layout(
        xaxis_title="Sensor Id",
        yaxis_title="Sensor reading for "+col.upper()+" ("+units+")",
        font=dict(
            size=12
        )
    )
    fig.show()

### Interactive outputs for dashboard

In [22]:
# display the widgets
ui

In [23]:
# fetch data
widgets.interactive_output(get_data,{'selected_city':select_city,'ndays':select_ndays})

Output()

In [24]:
# contour map
widgets.interactive_output(spatialVis1,{'selected_city':select_city, 'col':select_col})

Output()

In [25]:
# time series (single resource)
widgets.interactive_output(timeSeriesVis1,{'selected_city':select_city, 'col':select_col, 'ndays':select_ndays})

Output()

In [26]:
# bubble map
widgets.interactive_output(spatialVis2,{'selected_city':select_city, 'col':select_col})

Output()

In [27]:
# time series (for all resources)
widgets.interactive_output(timeSeriesVis2,{'selected_city':select_city, 'col':select_col, 'ndays':select_ndays})

Output()

In [28]:
# bar chart
widgets.interactive_output(simpleVis1,{'selected_city':select_city, 'col':select_col})

Output()

In [29]:
# box plots
widgets.interactive_output(timeSeriesVis3,{'selected_city':select_city, 'col':select_col, 'ndays':select_ndays})

Output()

In [30]:
# histogram
widgets.interactive_output(timeSeriesVis4,{'selected_city':select_city, 'col':select_col, 'ndays':select_ndays})

Output()

In [31]:
timeSeriesVis4('pune','co',2)

HTML(value='<center style="font-size:14px">Histogram for \n CO from 2021-03-28 to 2021-03-29 for resource at […